## Static HRTF Demo 
Preamble:
- Libraries and Dependencies
- Publicaly Available HRIR and HRTF sets
- File Structure

# Part 1
- Choose desired HRIR sets 
    - Visualization 
- Choose example source(s) to spatialize 
- Check 
    - Channels
    - Sample Rates 
- Convolve HRIR with example source(s)
- Add and normalize
- 3D audio mix for headphones 

# Part 2 
- Multi-source Binaural Display 
- Random Soundfield Display 
- SOFA version 


In [67]:
import numpy as np
import matplotlib.pyplot
import sys,glob
import soundfile as sf ## <-- to read the audio 
import sofa ##<-- read SOFA files 
import librosa ## <-- resample function 
from scipy import signal ##<-- fast convolution function 
from IPython.display import Audio ##<-- Audio listening in the notebook 
import streamlit

In [68]:
hrtf_dir_SOFA = '/Users/anishnair/Project_Studio_Tech/Spatial-Data/Sofa-Far-Field/*.sofa'
source_dir = '/Users/anishnair/Project_Studio_Tech/Spatial-Data/48k-Sounds/*.wav'
_SOURCES = glob.glob(source_dir)
_SOFA = glob.glob(hrtf_dir_SOFA)
print(_SOFA) #Load the SOFA file data

['/Users/anishnair/Project_Studio_Tech/Spatial-Data/Sofa-Far-Field/HRIR_FULL2DEG.sofa', '/Users/anishnair/Project_Studio_Tech/Spatial-Data/Sofa-Far-Field/HRIR_L2702.sofa', '/Users/anishnair/Project_Studio_Tech/Spatial-Data/Sofa-Far-Field/HRIR_CIRC360.sofa']


In [69]:
file_path_0 = _SOFA[0]
sofa_file = sofa.Database.open(_SOFA[0]).Metadata.dump() #Load Metadata of the first .sofa file data 

APIName: ARI SOFA API for Matlab/Octave
APIVersion: 1.1.1
ApplicationName: 
ApplicationVersion: 
Author: Benjamin Bernschütz
AuthorContact: mail benjamin.bernschuetz@fh-koeln.de, GSM +49 171 4176069
Comment: KU100-HRIRs / Sampling Grid: Gauss-Leg. 16020SP (89E/180A)
Conventions: SOFA
DataType: FIR
DatabaseName: THK
DateCreated: 2012-08-23 10:56:10
DateModified: 2020-03-26 14:18:02
EmitterDescription: Genelec 8260
History: Converted from the miro file format
License: CC 3.0 BY-SA
ListenerDescription: Neumann KU100
ListenerShortName: HRIR_FULL2DEG MKII
Organization: Technische Hochschule Koeln, Germany
Origin: http://audiogroup.web.th-koeln.de
ReceiverDescription: Neumann KU100; Internal RME Fireface UCX Preamps
References: 
RoomDescription: FHK ZW 8-4 (Anechoic Chamber) / avgAirTemp: 29.78 / avgRelHumidity: 43
RoomType: free field
SOFAConventions: SimpleFreeFieldHRIR
SOFAConventionsVersion: 1.0
SourceDescription: Genelec 8260
Title: HRIR
Version: 1.0


In [70]:
#Function to find the closest array member to a target value
def find_nearest(array,value): #find the nearest angle within the sofa dataset to a target angle I need
    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return array[idx], idx

In [71]:
#Set sofaset to the number of the desired set to change the HRTF data
def random_display(N=1, set_index = 0, target_fs = 48000, repetition = False):
    #*** Presents a random binaural display in the horizontal plane
    #
    #
    #
    #
    # Inputs:
    # N = # of sources/angles desired
    # set_index = desired HRTF dataset
    # target_fs = desired sampling rate, everything is resampled to this value
    # repetition = if True, can re-use some source samples
    #
    # Returns:
    #  Stereo3D: Binauralized audio data (2-channels for headphones)
    if N < 1 or N > 18:
        print('Error: N needs to be a number in between 1 and 18')
        return -1
    # Init
    HRTF = sofa.Database.open(_SOFA[set_index]) #opens HRTF dataset from SOFA file located in set_index in _SOFA list
    fs_H = HRTF.Data.SamplingRate.get_values()[0] #retreives original sampling rate of the HRTF values
    positions = HRTF.Source.Position.get_values(system = "spherical") #fetches spherical coordinates (source position) from the HRTF data
    angles = np.arange(0,360,10) # Generates an array of angles from 0 to 350 with steps of 10 (horizontal azimuth)
    elevations = [-45,0,45] #down, 0, up (3 total elevations)
    sources = np.arange(0, len(_SOURCES)) #generates a array of indices of the available audio source samples
    H = np.zeros([HRTF.Dimensions.N, 2]) #holds an array H to hold HRTF data for each source consisting of 2 channels
    Stereo3D = np.zeros([HRTF.Dimensions.N, 2]) #Initializes array Stereo3D to store final binauralized data

    print("Using HRTF set:" + _SOFA[set_index][9:-5] + '.')
    try:
        print('Source distance is', positions[0,2], 'meters\n')
    except:
        print("No distance information available")
    
    for n in range(N): #ensures that all of the sources are looped once
        #Random direction on horizontal plane 
        angle_idx = np.random.randint(0, len(angles)) #random angle index chosen from 0 to the bounds of the angle array
        angle = angles[angle_idx] #soecific angle of the angle_idx position within the angles array from (0-360 degrees)
        #Remove angle and front/back complementary from list for next iteration 
        angles = np.delete(angles, angle_idx) #removes the angles array to ensure it does not repeat in subsequent iterations
        if angle < 180:
            arc_dist = 90-angle
            complementary = 90 + arc_dist #calculates the angle and the complementary when the angle less than 180. The complementary angle is calculated opposite of 180 degrees
        else:
            arc_dist = 270 - angle# if angle is larger than 180 then it finds an angle from 270 degrees and find complementary angle similarly 
            complementary = 270 + arc_dist
        if arc_dist != 0: #checks if angle is calculated arc-distance is non 0. A zero arc means the arc distance is either 90 or 270 which means no complementary angle
            angles = np.delete(angles, np.where(angles==complementary)) #removes complementary angle so it doesnt appear in subsequent iterations
        
        #Database-specific format adjustments 
        angle_label = angle
        angle = 360-angle 
        if angle ==360:
            angle =0 #if angle reaches 360 degrees it reaches origin or 0 degrees
        
        #Randomize elevation 
        elev_idx = np.random.randint(0, len(elevations))
        elev = elevations[elev_idx] #uses the random index of elev_idx stored between 0 to and not including the elevation list

        [az, az_indices] = find_nearest(positions[:,0], angle)
        az_indices = np.where(positions[:,0] == az)[0]
        [el, el_indices] = find_nearest(positions[az_indices][:,1], elev)
        M_idx = az_indices[el_indices]
        H[:,0] = HRTF.Data.IR.get_values(indices = {'M': M_idx, "R":0, "E":0})
        H[:,1] = HRTF.Data.IR.get_values(indices = {'M': M_idx, "R":1, "E":0})
 # right side of the HRTF#retreiving impulse response data from the HRTF
        if fs_H != target_fs:
            H = librosa.core.resample(H.transpose(), fs_H, target_fs).transpose() #adjust sampling rate (resampled)
        
        #Pick random sources 
        source_id = np.random.choice(sources) #source id selected randomly from the source array 
        [x,fs_x] = sf.read(_SOURCES[source_id], always_2d= True) #x is audio, fs_x is the sampling rate of the audio datas. sf.read() reads the audio data
        if x.shape[1] > 1:
            x = np.mean(x, axis=1) #if the audio x has more than one channel, take the mean with axis = 1 and convert to mono 
        if not repetition:
            sources = np.delete(sources, np.where(sources ==source_id)) #source should not be used more than once, if so removes source_id from sources array
        if fs_H != target_fs:
            x = librosa.core.resample(x.transpose(), fs_H, target_fs).transpose() #adjust sampling rate (resampled)
        #Convolve and add LR signals to final array (general pointwise multiplication frequency domain)
        H_left = H[:,0][:, np.newaxis]
        rend_L = signal.fftconvolve(x, H_left, mode='full') #fft convolution with Left mono signal and left impulse response 
        rend_L = rend_L.squeeze() #match ndim size
        H_right = H[:,1][:, np.newaxis]
        rend_R = signal.fftconvolve(x, H_right, mode='full')#fft convolution with Right mono signal and right impulse response
        rend_R = rend_R.squeeze() #match ndim size
        M = np.max([np.abs(rend_L), np.abs(rend_R)]) #normalization factor to avoid clipping 
        if len(Stereo3D) < len(rend_L):
            
            diff = len(rend_L) - len(Stereo3D) #if len(Stereo3D) is < than len(rend_L), pad with zeros to reach same length
            Stereo3D = np.append(Stereo3D, np.zeros([diff,2]), 0)
        print("rend_L shape:", rend_L.shape)
        print("rend_R shape:", rend_R.shape)
        print("Stereo3D shape:", Stereo3D.shape)
        print("Slice length for Stereo3D:", len(Stereo3D)) 
        Stereo3D[0:len(rend_L), 0] += (rend_L/M)
        Stereo3D[0:len(rend_R), 1] += (rend_R/M) #Signal mixing with added Normalization  
        #Print operation 
        print("Source #" + str(n+1) + _SOURCES[source_id][12:-8] + "rendered at azimuth" + str(angle_label) + "and elevation" + str(elev))
    return Stereo3D

In [72]:
#Example use - WEAR HEADPHONES
fs = 48000
binaural = random_display(N=2, set_index = 2, target_fs = fs)
Audio(binaural.transpose(), rate=fs)


Using HRTF set:ishnair/Project_Studio_Tech/Spatial-Data/Sofa-Far-Field/HRIR_CIRC360.
Source distance is 3.25 meters

rend_L shape: (220627,)
rend_R shape: (220627,)
Stereo3D shape: (220627, 2)
Slice length for Stereo3D: 220627
Source #1nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/knocrendered at azimuth40and elevation0
rend_L shape: (220627,)
rend_R shape: (220627,)
Stereo3D shape: (220627, 2)
Slice length for Stereo3D: 220627
Source #2nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/harendered at azimuth90and elevation45


In [73]:
def manualDisplay(angle_pos, elev_pos, source_IDs, set_index = 0, target_fs = 48000):
    if len(angle_pos) != len(source_IDs) or len(elev_pos) != len(source_IDs):
        print("Error: All input lists must have the same length.")
        return None, None
    #
    # - This function lets you manually create a display with desired azimuth positions and sources
    #
    # Inputs: 
    # angle_pos: array of azimuth angle positions, clockwise convention 
    # source_ids: array of desires sources from reference folder (needs to be same length as angle_pos) since each source will be convolved with the corresponding angle at the same index 
    # set_index: Desired HRTF dataset
    # target_fs = the target sampling rate, everything is resampled to this value
    # 
    # Returns:
    # Stereo 3D: Binauralized audio data (2-channels) per headphones
    if len(angle_pos) != len(source_IDs):
        print("Error the angle position and the source ID's must be of same dimensions")
    if len(elev_pos) != len(source_IDs):
        print("Error: the elevation position and the source ID's must be of the same dimension")
    
    #Init 
    HRTF = sofa.Database.open(_SOFA[set_index]) #opens HRTF data from HRTF dataset found in the sofa file _SOFA(set_index)
    fs_H = HRTF.Data.SamplingRate.get_values()[0] #retreives original sampling rate from the HRTF values
    positions = HRTF.Source.Position.get_values(system ="spherical") #fetches spherical coordinates (source position) from the HRTF data
    source = source_IDs
    H = np.zeros([HRTF.Dimensions.N, 2]) #Will store the HRTF impulse responses (IR) for the left and right ears 
    Stereo_3D = np.zeros([HRTF.Dimensions.N, 2]) #Will accumulate the final binaural audio output 

    print("Using HRTF set:" + _SOFA[set_index][9:-5] + '.') #prints the actual HRTF being used 
    try: 
        print("Source Distance Is" , positions[0,2], "meters\n") #prints the azimuth, elevation, and distance values]
    except:
        print("No distance information available")
    
    for n in range (len(angle_pos)):
        angle = angle_pos[n]
        elev = elev_pos[n]

        #Database Specific Requirements
        angle_label = angle
        angle = 360 - angle 
        if angle == 360:
            angle = 0
        #Retreive HRTF data for the angle
        [az, az_idx] = find_nearest(positions[:,0], angle)
        subpositions = positions[np.where(positions[:,0] == az)]
        [el, sub_idx] = find_nearest(subpositions[:,1], elev)

        M_idx = np.where((positions[:, 0] == az) & (positions[:, 1] == el))[0][0]
        H[:,0] = HRTF.Data.IR.get_values(indices = {"M": M_idx, "R":0, "E": 0})
        H[:,1] = HRTF.Data.IR.get_values(indices = {"M": M_idx, "R":1, "E": 0})
        if fs_H != target_fs:
            H = librosa.core.resample(H.transpose(), fs_H, target_fs).transpose()
        
        #Pick source
        [x, fs_x] = sf.read(_SOURCES[source[n]], always_2d= False)
        if x.ndim > 1:
             x= np.mean(x, axis = 1) #Average the signals to get a mono signal 
        if fs_x != target_fs:
            x = librosa.resample(x, orig_sr=fs_x, target_sr=target_fs)
        
        #Convolve and add signals to final array (general pointwise multiplication)
        H_left = H[:,0] #keep left side HRIR as a one dimensional array
        rend_L = signal.fftconvolve(x, H_left, mode='full') #fft convolution with Left mono signal and left impulse response 
        rend_L = rend_L.squeeze() #match ndim size
        H_right = H[:,1] #keep right side HRIR one dimensional array
        rend_R = signal.fftconvolve(x, H_right, mode='full')#fft convolution with Right mono signal and right impulse response
        rend_R = rend_R.squeeze() #match ndim size
        M = max(np.max(np.abs(rend_L)), np.max(np.abs(rend_R))) #normalization factor to avoid clipping (scalar value)
        print(M)
        if len(Stereo_3D) < len(rend_L):
            
            diff = len(rend_L) - len(Stereo_3D) #if len(Stereo3D) is < than len(rend_L), pad with zeros to reach same length
            Stereo_3D = np.append(Stereo_3D, np.zeros([diff,2]), 0)
        print("rend_L shape:", rend_L.shape)
        print("rend_R shape:", rend_R.shape)
        print("Stereo3D shape:", Stereo_3D.shape)
        print("Slice length for Stereo3D:", len(Stereo_3D)) 
        Stereo_3D[0:len(rend_L), 0] += (rend_L/M)
        Stereo_3D[0:len(rend_R), 1] += (rend_R/M) #Signal mixing with added Normalization  
        #Print operation 
        print("Source #" + str(n+1) + _SOURCES[source[n]][12:-4] + "rendered at azimuth" + str(angle_label) + "and elevation" + str(elev))
    
    return Stereo_3D, positions

In [74]:
## It helps to know the order of the source sample files in the folder 
for c, s in enumerate(_SOURCES):
    print("#" + str(c) + " is source: " + s[12:-4])

#0 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/birds2
#1 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/vaccum
#2 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/helicopter
#3 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/bark
#4 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/gunshot
#5 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/raining
#6 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/engine
#7 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/train
#8 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/saw
#9 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/helicopter2
#10 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/pigs
#11 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/fire
#12 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/typing
#13 is source: nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/can_open
#14 is source

In [80]:
fs = 48000
binaural_manual, POS = manualDisplay([290, 180, 270], [180, 20, -60], [8, 3, 14], 1) #(azimuth), (elevation), (source), (sofa_file)
Audio(binaural_manual.transpose(), rate =fs)


Using HRTF set:ishnair/Project_Studio_Tech/Spatial-Data/Sofa-Far-Field/HRIR_L2702.
Source Distance Is 3.25 meters

0.22233643738455025
rend_L shape: (240128,)
rend_R shape: (240128,)
Stereo3D shape: (240128, 2)
Slice length for Stereo3D: 240128
Source #1nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/sawrendered at azimuth290and elevation180
1.0255131576203138
rend_L shape: (240128,)
rend_R shape: (240128,)
Stereo3D shape: (240128, 2)
Slice length for Stereo3D: 240128
Source #2nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/barkrendered at azimuth180and elevation20
1.5773748210123235
rend_L shape: (240128,)
rend_R shape: (240128,)
Stereo3D shape: (240128, 2)
Slice length for Stereo3D: 240128
Source #3nair/Project_Studio_Tech/Spatial-Data/48k-Sounds/planerendered at azimuth270and elevation-60
